# dependancys 

In [ ]:
from itertools import combinations_with_replacement
from collections import defaultdict
import cv2
import numpy as np
from numpy.linalg import inv
from google.colab.patches import cv2_imshow
import pandas as pd 
import glob
import os 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Histogram equlization 

> is the way we choose to equlize the intencety of the images, [OpenCV](https://docs.opencv.org/3.4/d4/d1b/tutorial_histogram_equalization.html) has Impalimentaion of it wich handles gray images, so acording to this  [StackoverFlow post](https://stackoverflow.com/questions/31998428/opencv-python-equalizehist-colored-image#38312281) in RGB images each chanale represents the **intinsity of the related color** so we saprated the **Brightness** of the image, By using **standardized** colorspaces that encode brightness and color **separately** like (YCbCr, HSV, ect ). 
![](https://github.com/AbdoAli05/Scene-Understanding-in-Low-Light-Conditions/blob/main/Images/mermaid-diagram-20220531011520.png?raw=true)
and then **Applay simple Histogram**, because Histogram lacking results, due to it's lack of ragrad to outliars and the location of the pixel, we chose to implemint Contrast Limited Adaptive HE witch have better slitly better reusltus.

## results 


>![before](https://github.com/AbdoAli05/Scene-Understanding-in-Low-Light-Conditions/blob/main/Images/befortype3.png?raw=true)

>![After](https://github.com/AbdoAli05/Scene-Understanding-in-Low-Light-Conditions/blob/main/Images/befortype3.png?raw=true)

>![before](https://github.com/AbdoAli05/Scene-Understanding-in-Low-Light-Conditions/blob/main/Images/type3befor2.png?raw=true)

>![after](https://github.com/AbdoAli05/Scene-Understanding-in-Low-Light-Conditions/blob/main/Images/type3after2.png?raw=true)

--- 


# Filters and Sparte Chanales

In [ ]:

# FILters and Sprate Chanles. 
clip=6.0
def equalize_clahe_color_hsv(img):
  cla = cv2.createCLAHE(clipLimit=clip) # creatin Object with clip ie. constract the defult is 45
  H, S, V = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2HSV))
  eq_V = cla.apply(V)
  eq_image = cv2.cvtColor(cv2.merge([H, S, eq_V]), cv2.COLOR_HSV2BGR)
  return eq_image
def equalize_clahe_color_lab(img):
  cla = cv2.createCLAHE(clipLimit=clip)
  L, a, b = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2Lab))
  eq_L = cla.apply(L)
  eq_image = cv2.cvtColor(cv2.merge([eq_L, a, b]), cv2.COLOR_Lab2BGR)
  return eq_image
def equalize_clahe_color_yuv(img):
  cla = cv2.createCLAHE(clipLimit=clip)
  Y, U, V = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2YUV))
  eq_Y = cla.apply(Y)
  eq_image = cv2.cvtColor(cv2.merge([eq_Y, U, V]), cv2.COLOR_YUV2BGR)
  return eq_image
def equalize_clahe_color(img):
  cla = cv2.createCLAHE(clipLimit=clip)
  channels = cv2.split(img)
  eq_channels = []
  for ch in channels:
      eq_channels.append(cla.apply(ch))
      eq_image = cv2.merge(eq_channels)
  return eq_image

def equalize_cvtColor(img):
  img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

  # Histogram equalisation on the V-channel
  img_hsv[:, :, 2] = cv2.equalizeHist(img_hsv[:, :, 2])
  img2 = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
  return img2



# Loading Images based on Light types 

os.chdir("/content/drive/MyDrive/Cv_ProjectData/Images/Images") # Changing dir to where the Images are in the Drive. 
path = "/content/drive/MyDrive/Cv_ProjectData/Fillterd/train/" # path saving Filterd Images to 

imageLabel = pd.read_csv('/content/drive/MyDrive/Cv_ProjectData/TrainImageLabels.txt',sep = " ") # Reading Images Labels. 

# Loading Rows of each image light type  
image1 = imageLabel.loc[imageLabel["LightType"] == 1]
image2 = imageLabel.loc[imageLabel["LightType"] == 2]
image3 = imageLabel.loc[imageLabel["LightType"] == 3]
image4 = imageLabel.loc[imageLabel["LightType"] == 4]
image5 = imageLabel.loc[imageLabel["LightType"] == 5]
image6 = imageLabel.loc[imageLabel["LightType"] == 6]
image7 = imageLabel.loc[imageLabel["LightType"] == 7]
image8 = imageLabel.loc[imageLabel["LightType"] == 8]
image9 = imageLabel.loc[imageLabel["LightType"] == 9]
image10 = imageLabel.loc[imageLabel["LightType"] == 10]
# Filtring Rows and keeping only the pass of the intendet Images. 
image1Name = image1["ImageName"]
image2Name = image2["ImageName"]
image3Name = image3["ImageName"]
image4Name = image4["ImageName"]
image5Name = image5["ImageName"]
image6Name = image6["ImageName"]
image7Name = image7["ImageName"]
image8Name = image8["ImageName"]
image9Name = image9["ImageName"]
image10Name =image10["ImageName"]
# converting data fram to list 
image1Name = image1Name.to_list()
image2Name = image2Name.to_list()
image3Name = image3Name.to_list()
image4Name = image4Name.to_list()
image5Name = image5Name.to_list()
image6Name = image6Name.to_list()
image7Name = image7Name.to_list()
image8Name = image8Name.to_list()
image9Name = image9Name.to_list()
image10Name = image10Name.to_list()

# calling Fillters By Itrating on every Image 
## Type one 
count = 5 # count of images keep it 1 to itrate over all images
for i in image1Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_hsv(img) # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 1 ")
    print()
    print()
    break

         

## Type two 
count = 5 # count of images keep it 1 to itrate over all images
for i in image2Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img= equalize_cvtColor(img)  # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 2 ")
    print()
    print()
    break

## Type three 
count = 5 # count of images keep it 1 to itrate over all images
for i in image3Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_hsv(img) # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 1 ")
    print()
    print()
    break

## Type four 
count = 5 # count of images keep it 1 to itrate over all images
for i in image4Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_yuv(img)  # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if  not count:
    print("IMage type 4 ")
    print()
    print()
    break

## Type five 
count = 5 # count of images keep it 1 to itrate over all images
for i in image5Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color(img)  # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 5 ")
    print()
    print()
    break

## Type six  
count = 5 # count of images keep it 1 to itrate over all images
for i in image6Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_hsv(img) # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 6 ")
    print()
    print()
    break

## Type siven  
count = 5 # count of images keep it 1 to itrate over all images
for i in image7Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_hsv(img) # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 7 ")
    print()
    print()
    break

## Type eight  
count = 5 # count of images keep it 1 to itrate over all images
for i in image8Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_hsv(img) # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 8 ")
    print()
    print()
    break

## Type nine 
count = 5 # count of images keep it 1 to itrate over all images
for i in image9Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_hsv(img) # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 9 ")
    print()
    print()
    break

## Type ten 
count = 5 # count of images keep it 1 to itrate over all images
for i in image10Name : 
  img = cv2.imread(i) # Reading Images 
  cv2_imshow(img) # showing image befor
  img = equalize_clahe_color_lab(img) # Equlizing the image 
  cv2_imshow(img) # showing image after enhancement
  # cv2.imwrite(path+i , img) # writing the filterd image into dis ps concatinating to save with rigth name
  count-=1 # counter for pringting 5 image only and converting 5 
  if not count:
    print("IMage type 10 ")
    print()
    print()
    break





Output hidden; open in https://colab.research.google.com to view.